# Heat Flow Analysis
This notebook performs the heat flow analysis and anomaly quantification for the study area.
It is the main notebook for a regional heat flow analyis.

In [ ]:
import numpy as np
from pyproj import Proj
from cache import cached_call
import matplotlib.pyplot as plt
from flottekarte import Map, GeoJSON
from pickle import Pickler, Unpickler
from reheatfunq import GammaConjugatePrior, AnomalyLS1980, HeatFlowAnomalyPosterior, HeatFlowPredictive
from loaducerf3 import PolygonSelector, Polygon
#from zeal2022hf import LS_a

Configure plots to look good on a HiDPI monitor (you may not need the following configuration if you are not using a HiDPI monitor):

In [ ]:
%config InlineBackend.figure_format = 'retina'

## Data
First we load the data from the previous notebooks.

First the gamma conjugate prior parameters:

In [ ]:
PRIOR_P, PRIOR_S, PRIOR_N, PRIOR_V = np.loadtxt("results/05-GCP-Parameters.txt", skiprows=1, delimiter=',')
gcp = GammaConjugatePrior(PRIOR_P, PRIOR_S, PRIOR_N, PRIOR_V)

Geometry. If you have adapted the loading of the fault trace (e.g. for another area),
you can replace loading the `merged_surface_trace` from `intermediate/05-SoCal-UCERF3-default-branches.pickle`
with custom code.

In [ ]:
with open('intermediate/02-Geometry.pickle','rb') as f:
    geometry = Unpickler(f).load()
with open('intermediate/05-SoCal-UCERF3-default-branches.pickle','rb') as f:
    ucerf3_res = Unpickler(f).load()
    merged_surface_trace = ucerf3_res["merged_surface_trace"]
    fault_depth = ucerf3_res["d"]

In [ ]:
proj_str = geometry["proj_str"]
ENCOS_poly_xy = geometry["encos_poly_xy"]
selection_polys_xy = geometry["selection_polygons_xy"]
proj = Proj(proj_str)

Heat flow data

In [ ]:
hf_continental_nghf_mW_m2 = np.load('intermediate/heat-flow-selection-mW_m2.npy')
hf_x, hf_y = proj(hf_continental_nghf_mW_m2[1], hf_continental_nghf_mW_m2[2])
hf_xy = np.stack((hf_x,hf_y), axis=1)
hf_mW_m2 = hf_continental_nghf_mW_m2[0]
hf_xyq_mW_m2 = np.stack((hf_x, hf_y, hf_mW_m2), axis=1)

Load the coastline:

#### Data Selection
Here, we 

In [ ]:
NAMES = ["Mojave", "", "", ""]

In [ ]:
selectors = [PolygonSelector(Polygon(*poly[:-1,:].T)) for poly in selection_polys_xy]

In [ ]:
xy_ano = [merged_surface_trace[sel.array_mask(merged_surface_trace)] for sel in selectors]
anomalies = [AnomalyLS1980(xya, d) for xya,d in zip(xy_ano,fault_depth)]

In [ ]:
hf_select = []
hf_xy_select = []
ci_select = []

fig = plt.figure(figsize=(12,12))
for i in range(len(selectors)):
    mask_i = selectors[i].array_mask(hf_xy)
    
    hf_xy_i = hf_xy[mask_i,:]
    hf_i = hf_mW_m2[mask_i]
    ci = anomalies[i](hf_xy_i)
    
    hf_select.append(hf_i)
    hf_xy_select.append(hf_xy_i)
    ci_select.append(ci)

    ax = fig.add_subplot(2,2,i+1)
    ax.set_aspect('equal')
    ax.scatter(*hf_xy[mask_i,:].T, c=ci)
    ax.plot(*xy_ano[i].T)
    ax.set_title(NAMES[i])

fig.tight_layout()

## Heat Flow Distribution
First, evaluate the posterior predictive distributions within the regions:

In [ ]:
DMIN = 20e3

In [ ]:
hfp = [HeatFlowPredictive(hf_i, *hf_xy_i.T, gcp, dmin=DMIN, n_bootstrap=1000)
       for hf_i, hf_xy_i in zip(hf_select, hf_xy_select)]

In [ ]:
q = np.linspace(0, 150, 200)

In [ ]:
from math import exp

for i in range(len(hfp)):
    y = hfp[i].cdf(q)
    y2 = hfp[i].pdf(q)
    hf_i = hf_select[i]
    hf_xy_i = hf_xy_select[i]

    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(121)
    ax.plot(q, y, color='k')
    ax.step(list(sorted(hf_i)), list((np.arange(hf_i.size)+1) / hf_i.size), where='post', color='tab:orange')
    for w,ids in hfp[i].bootstrap:
        ax.plot(list(sorted(hf_i[ids])), list((np.arange(ids.size)+1) / ids.size),
                color='bisque', zorder=0)
    ax = fig.add_subplot(122)
    ax.hist(hf_i, zorder=0, density=True, color='bisque')
    ax.plot(q, y2, color='k', linewidth=2)
    for lp,s,n,v in zip(hfp[i].lp, hfp[i].s, hfp[i].n, hfp[i].v):
        ax.plot(q, GammaConjugatePrior(exp(lp), s, n, v).posterior_predictive(q),
                color='steelblue', zorder=0, linewidth=0.5)
    

##### Notes:
 - (1) Mojave section is a very good example of uniform and sufficient data distribution!
 - (2) ?? section has sufficient data. It shows clustering: the combined CDF is significantly
   shifted compared to the distributions with $d_\mathrm{min}=20\,\mathrm{km}$.
 - (3) Too few data - only three different combinations! (two data points have same heat flow,
   so that two distributions overlay).
 - (4) Too few data. Clustering at the Geysers leading to two distinct distributions?

In [ ]:
hfp[2].bootstrap

In [ ]:
hfp[2].q

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_aspect('equal')
ax.scatter(*hf_xy_select[3].T, c=hf_select[3])

## Heat Flow Anomaly Analysis

In [ ]:
hfap = [HeatFlowAnomalyPosterior(hf_i, *hf_xy_i.T, anomaly, gcp, dmin=DMIN, n_bootstrap=100)
        for hf_i, hf_xy_i, anomaly in zip(hf_select, hf_xy_select, anomalies)]

In [ ]:
P_H = [np.linspace(0, hfap[i].PHmax, 200) for i in range(len(hfap))]

In [ ]:
[hf.PHmax for hf in hfap]

In [ ]:
y  = [hfap[i].pdf(P_H[i]) for i in range(len(hfap))]
y2 = [hfap[i].cdf(P_H[i]) for i in range(len(hfap))]
y3 = [hfap[i].tail(P_H[i]) for i in range(len(hfap))]

In [ ]:
fig = plt.figure(figsize=(12,12))
for i in range(len(hfap)):
    ax = fig.add_subplot(2,2,i+1)
    ax.plot(P_H[i], y[i])
    twax = ax.twinx()
    twax.plot(P_H[i], y2[i], linestyle=':')
    twax.plot(P_H[i], y3[i], linestyle='--')

### License
```
A notebook to compute radiated energy, fault geometry, and slip
rates from the UCERF3 model branches.

This file is part of the REHEATFUNQ model.

Author: Malte J. Ziebarth (ziebarth@gfz-potsdam.de)

Copyright © 2019-2022 Deutsches GeoForschungsZentrum Potsdam,
            2022 Malte J. Ziebarth
            

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
```